# DSCI 525 - Web and Cloud Computing
## Milestone 1: Tackling big data on your laptop
### Group 14
Group Members: Sasha Babicki, Cheuk Ho, Sakshi Jain, Zeliha Ural Merpez

#### 3. Download the data
1. Download the data from figshare to your local computer using the figshare API (you can make use of requests library).
2. Extract the zip file, again programmatically, similar to how we did it in class.

#### Note: code below is modified from 525 lecture notes
https://github.ubc.ca/MDS-2020-21/DSCI_525_web-cloud-comp_students/blob/master/Lectures/Lecture_1_2.ipynb

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
from memory_profiler import memory_usage

In [2]:
%load_ext rpy2.ipython
%load_ext memory_profiler

In [3]:
article_id = 14096681  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figshareairline/"

In [4]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'is_link_only': False,
  'name': 'daily_rainfall_2014.png',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'id': 26579150,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'size': 58863},
 {'is_link_only': False,
  'name': 'environment.yml',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'id': 26579171,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'size': 192},
 {'is_link_only': False,
  'name': 'README.md',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'id': 26586554,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'size': 5422},
 {'is_link_only': False,
  'name': 'data.zip',
  'supplied_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'computed_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'id': 26766812,
  'download_url': 'https://

In [5]:
%%time

download_file = "data.zip"
for file in files:
    if file["name"] == download_file:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 4.72 s, sys: 4.17 s, total: 8.89 s
Wall time: 1min 3s


In [6]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, download_file), 'r') as f:
    f.extractall(output_directory)

CPU times: user 17 s, sys: 3.58 s, total: 20.6 s
Wall time: 25.1 s


#### 4. Combining data CSVs
1. Use one of the following options to combine data CSVs into a single CSV. (Pandas, DASK)
2. When combining the csv files make sure to add extra column called "model" that identifies the model (tip : you can get this column populated from the file name eg: for file name "SAM0-UNICON_daily_rainfall_NSW.csv", the model name is SAM0-UNICON)
3. Compare run times and memory usages of these options on different machines within your team, and summarize your observations in your milestone notebook.

In [7]:
combined_file_path = output_directory + "combined_data.csv"

In [8]:
%%time
%memit

files = glob.glob(output_directory + "*_daily_rainfall_NSW.csv")
df = pd.concat(
    (
        pd.read_csv(file, index_col=0).assign(
            model=re.findall(r"/(.*)_daily_rainfall", file)[0]
        )
        for file in files
    )
)
df.to_csv(combined_file_path)

peak memory: 148.91 MiB, increment: 0.20 MiB
CPU times: user 6min 35s, sys: 17.2 s, total: 6min 52s
Wall time: 7min 38s


In [9]:
%%sh
du -sh figshareairline/combined_data.csv

5.6G	figshareairline/combined_data.csv


In [10]:
%%time
df = pd.read_csv(combined_file_path, index_col=0, parse_dates=True)

CPU times: user 1min 10s, sys: 14.4 s, total: 1min 24s
Wall time: 1min 32s


In [11]:
print(df.shape)

(62467843, 6)


In [12]:
%%time
df

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 6.2 µs


,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,MPI-ESM-1-2-HAM
1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,MPI-ESM-1-2-HAM
1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,MPI-ESM-1-2-HAM
1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,MPI-ESM-1-2-HAM
1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,MPI-ESM-1-2-HAM
...,...,...,...,...,...,...
2014-12-27 12:00:00,-30.157068,-29.214660,153.1250,154.3750,6.689683e+00,SAM0-UNICON
2014-12-28 12:00:00,-30.157068,-29.214660,153.1250,154.3750,7.862555e+00,SAM0-UNICON
2014-12-29 12:00:00,-30.157068,-29.214660,153.1250,154.3750,1.000503e+01,SAM0-UNICON


In [13]:
df['model'].nunique()

27

In [14]:
df['model'].unique()

array(['MPI-ESM-1-2-HAM', 'AWI-ESM-1-1-LR', 'NorESM2-LM', 'ACCESS-CM2',
       'FGOALS-f3-L', 'CMCC-CM2-HR4', 'MRI-ESM2-0', 'GFDL-CM4',
       'BCC-CSM2-MR', 'EC-Earth3-Veg-LR', 'CMCC-ESM2', 'NESM3',
       'MPI-ESM1-2-LR', 'ACCESS-ESM1-5', 'FGOALS-g3', 'INM-CM4-8',
       'MPI-ESM1-2-HR', 'TaiESM1', 'NorESM2-MM', 'CMCC-CM2-SR5',
       'KIOST-ESM', 'INM-CM5-0', 'MIROC6', 'BCC-ESM1', 'GFDL-ESM4',
       'CanESM5', 'SAM0-UNICON'], dtype=object)

In [15]:
%%time
df.describe()

CPU times: user 12.6 s, sys: 6.13 s, total: 18.7 s
Wall time: 18.9 s


,lat_min,lat_max,lon_min,lon_max,rain (mm/day)
count,5.924854e+07,6.246784e+07,5.924854e+07,6.246784e+07,5.924854e+07
mean,-3.310482e+01,-3.197757e+01,1.469059e+02,1.482150e+02,1.901170e+00
std,1.963549e+00,1.992067e+00,3.793784e+00,3.809994e+00,5.585735e+00
min,-3.646739e+01,-3.600000e+01,1.406250e+02,1.412500e+02,-3.807373e-12
25%,-3.486911e+01,-3.366221e+01,1.434375e+02,1.450000e+02,3.838413e-06
50%,-3.300000e+01,-3.204188e+01,1.468750e+02,1.481250e+02,6.154947e-02
75%,-3.140170e+01,-3.015707e+01,1.501875e+02,1.513125e+02,1.020918e+00
max,-2.990000e+01,-2.790606e+01,1.537500e+02,1.556250e+02,4.329395e+02


##### 4.3 Runtime Observations: 

##### Zeliha
- Combining files:
    - peak memory: 13663.16 MiB, increment: 0.01 MiB
    - CPU times: user 4min 16s, sys: 6.37 s, total: 4min 22s
    - Wall time: 4min 24s
- Reading combined file:
    - CPU times: user 45.3 s, sys: 3.61 s, total: 48.9 s
    - Wall time: 49 s
    
##### Sasha
- Combining files:
    - peak memory: 86.82 MiB, increment: 0.26 MiB
    - CPU times: user 6min 14s, sys: 2min 41s, total: 8min 56s
    - Wall time: 9min 26s
- Reading combined file:
    - CPU times: user 1min 8s, sys: 15.3 s, total: 1min 23s
    - Wall time: 1min 35s
    
##### Chuck
- Combining files:
    - peak memory: 75.36 MiB, increment: 0.71 MiB
    - CPU times: user 5min 59s, sys: 15.5 s, total: 6min 15s
    - Wall time: 6min 22s
- Reading combined file:
    - Peak memory: 3355.27 MiB, increment: 0.12 MiB
    - CPU times: user 56.9 s, sys: 14.2 s, total: 1min 11s
    - Wall time: 1min 14s

### 5. Load the combined CSV to memory and perform a simple EDA
1. Investigate at least two of the following approaches to reduce memory usage while performing the EDA (e.g., value_counts).
    - Changing dtype of your data
    - Load just columns what we want
    - Loading in chunks
    - Dask
2. Discuss your observations.

#### 5.1.1 Changing dtype of data:

In [16]:
# View original dtypes
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 62467843 entries, 1889-01-01 12:00:00 to 2014-12-31 12:00:00
Data columns (total 6 columns):
 #   Column         Dtype  
---  ------         -----  
 0   lat_min        float64
 1   lat_max        float64
 2   lon_min        float64
 3   lon_max        float64
 4   rain (mm/day)  float64
 5   model          object 
dtypes: float64(5), object(1)
memory usage: 3.3+ GB


In [17]:
float_cols = ["lat_min","lat_max","lon_min","lon_max","rain (mm/day)"]
df_32 = df.copy()
df_64 = df.copy()

df_32[float_cols] = df_32[float_cols].astype('float32', errors='ignore')
print(f"DataFrame with numeric columns as float64: {df_64.memory_usage().sum() / 1e9:.2f} GB")
print(f"DataFrame with numeric columns as float32: {df_32.memory_usage().sum() / 1e9:.2f} GB")

DataFrame with numeric columns as float64: 3.50 GB
DataFrame with numeric columns as float32: 2.25 GB


In [18]:
%%time
%%memit
df_64["lat_min"].value_counts()

peak memory: 4925.93 MiB, increment: 484.40 MiB
CPU times: user 713 ms, sys: 400 ms, total: 1.11 s
Wall time: 3.37 s


In [19]:
%%time
%%memit
df_32["lat_min"].value_counts()

peak memory: 5403.10 MiB, increment: 477.14 MiB
CPU times: user 758 ms, sys: 163 ms, total: 921 ms
Wall time: 1.49 s


#### 5.1.2 Dask:

In [20]:
import dask.dataframe as dd

df_dask = dd.read_csv(combined_file_path)

In [21]:
%%time
%%memit
df["lat_min"].value_counts()

peak memory: 4814.78 MiB, increment: 67.14 MiB
CPU times: user 648 ms, sys: 29.7 ms, total: 677 ms
Wall time: 1.23 s


In [22]:
%%time
%%memit
df_dask["lat_min"].value_counts()

peak memory: 4814.86 MiB, increment: 0.07 MiB
CPU times: user 52.5 ms, sys: 23.7 ms, total: 76.2 ms
Wall time: 1.5 s


In [23]:
%%time
%%memit
df["model"].value_counts()

peak memory: 4874.87 MiB, increment: 60.00 MiB
CPU times: user 4.75 s, sys: 56.4 ms, total: 4.8 s
Wall time: 5.36 s


In [24]:
%%time
%%memit
df_dask["model"].value_counts()

peak memory: 4874.88 MiB, increment: 0.00 MiB
CPU times: user 52.2 ms, sys: 20.5 ms, total: 72.7 ms
Wall time: 1.55 s


#### 5.2 Discussion:

- Changing the dtype of numeric columns from `float64` to `float32` did reduce the space the dataframe takes in memory by almost half. However, performing `value_counts()` on a column actually used more memory and was slower for `float32` columns than `float64` columns. 
- Dask seems to use marginally less memory and is slightly faster when performing `value_counts()` on a numeric column. It is significantly faster when operating on a `str` type column.

#### 6. Perform a simple EDA in R
1. Pick an approach to transfer the dataframe from python to R.
    - Parquet file
    - Feather file
    - Pandas exchange
    - Arrow exchange
2. Discuss why you chose this approach over others.

In [25]:
## Install the pyarrow packages: https://arrow.apache.org/docs/python/install.html
import pyarrow.dataset as ds
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.feather as feather

## Install rpy2: https://anaconda.org/conda-forge/rpy2
import rpy2.rinterface

In [26]:
%%time
%%memit
table = pa.Table.from_pandas(df)

peak memory: 6153.34 MiB, increment: 1271.64 MiB
CPU times: user 4.5 s, sys: 509 ms, total: 5.01 s
Wall time: 3.66 s


In [27]:
%%time

# Write to feather format 
feather.write_feather(table, "figshareairline/example.feather")

CPU times: user 2.33 s, sys: 1.6 s, total: 3.93 s
Wall time: 4.61 s


In [28]:
%%sh
du -sh figshareairline/example.feather

1.1G	figshareairline/example.feather


In [29]:
%%time
%%R
### her we are showing how much time it took to read a feather file what we wrote in python
library(arrow)
library(dplyr)

start_time <- Sys.time()
r_table <- arrow::read_feather("figshareairline/example.feather")
print(class(r_table))

result <- r_table %>% count(model)
end_time <- Sys.time()
print(result)
print(end_time - start_time)

R[write to console]: 
Attaching package: ‘arrow’


R[write to console]: The following object is masked from ‘package:utils’:

    timestamp


R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




[1] "tbl_df"     "tbl"        "data.frame"
# A tibble: 27 x 2
   model                  n
   <chr>              <int>
 1 ACCESS-CM2       1932840
 2 ACCESS-ESM1-5    1610700
 3 AWI-ESM-1-1-LR    966420
 4 BCC-CSM2-MR      3035340
 5 BCC-ESM1          551880
 6 CanESM5           551880
 7 CMCC-CM2-HR4     3541230
 8 CMCC-CM2-SR5     3541230
 9 CMCC-ESM2        3541230
10 EC-Earth3-Veg-LR 3037320
# … with 17 more rows
Time difference of 27.55319 secs
CPU times: user 11.2 s, sys: 11.2 s, total: 22.3 s
Wall time: 29.2 s


#### 6.2 Discussion: 
- We choose to get table directly from pandas data frame and then save the file as feather. Both constructing table and saving file was quite fast.
- Using the feather file format saves a lot of space, the file only takes up ~1.2 GB in memory.